Practice - Loan Data set from Dream Housing Finance Company in all home loans
1. Use Python pandas(NumPy, Pandas, Sklearn)
2. Logistic Regression model

 **Reference**

deal with data（pandas）：
（data type）https://www.cnblogs.com/onemorepoint/p/9404753.html
https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/
infer_objects: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html#pandas.to_numeric
to_numerice: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html#pandas.to_numeric



Missing value：
（kaggle）https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
（normal methods）https://zhuanlan.zhihu.com/p/21549898
（review later）https://clevertap.com/blog/how-to-treat-missing-values-in-your-data-part-ii/
（data mining）https://blog.csdn.net/lujiandong1/article/details/52654703


SAMPLE
https://machinelearningmastery.com/handle-missing-data-python/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

**1. Anaylze data characters**

In [ ]:
#insert data
df = pd.read_csv('../input/loan_data_set.csv')
#infer_objects()
df=df.infer_objects()
df.dtypes

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
#for stripe, obeserve the distribution
df['Property_Area'].value_counts()

**2. The Distribution of Variables**

2.1 Applicant income

In [ ]:
df['ApplicantIncome'].hist(bins=60)
plt.show()
df.boxplot(column='ApplicantIncome')
plt.show()
#compared with Education
df.boxplot(column='ApplicantIncome',by='Education')
plt.show()

2.2 Loan Amonut

In [ ]:
df['LoanAmount'].hist(bins=60)
plt.show()
df.boxplot(column='LoanAmount')
plt.show()

2.3 Credit History

In [ ]:
temp1 = df['Credit_History'].value_counts(ascending=True)
print ('Frequency Table for Credit History')
print(temp1)
temp1.plot(kind='bar')

3. deal with data

In [ ]:
df.isnull().sum()

In [ ]:
#impute object with mode - fill NaN
df['Gender'].fillna('Male',inplace=True)
df['Married'].fillna('Yes',inplace=True)
df['Dependents'].fillna('0',inplace=True)
df['Self_Employed'].fillna('No',inplace=True)
df['Credit_History'].fillna('1.0',inplace=True)
df.isnull().sum()

In [ ]:
#impute int(float) with mean
df.fillna(df['LoanAmount'].mean(),inplace=True)
df.fillna(df['Loan_Amount_Term'].mean(),inplace=True)
df.isnull().sum()

In [ ]:
#create TotalIncome = ApplicantIncome + CoapplicantIncome
df['TotalIncome']=df['ApplicantIncome']+df['CoapplicantIncome']
df['TotalIncome_log']=np.log(df['TotalIncome'])

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
#transfer bool to float
df.dtypes
df["Married"]=np.where(df["Married"] == "Yes",1,0)
df["Gender"]=np.where(df["Gender"] == "Male",1,0)
df["Education"]=np.where(df["Education"] == "Graduate",1,0)
df["Self_Employed"]=np.where(df["Self_Employed"] == "Yes",1,0)
df["Loan_Status"]=np.where(df["Loan_Status"] == "Y",1,0)
df["Dependents"]=np.where(df["Dependents"] == "3+",3,df["Dependents"])
df["Dependents"]=df["Dependents"].astype("float")
df.head()

In [ ]:
def fun(x):
    if x == 'Rural':
        return 1
    elif x == 'Urban':
        return 2
    else:
        return 3

df['Property_Area'] = df.Property_Area.apply(lambda x: fun(x))

In [ ]:
#check types of data
print(df.head(5))
print('-'*50)
df.dtypes


4. Build model

In [ ]:
from sklearn import metrics
X=df[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'TotalIncome_log','LoanAmount',
       'Loan_Amount_Term','Credit_History','Property_Area']]
Y=df['Loan_Status']
from sklearn.model_selection import train_test_split
X_trainL,X_testL,Y_trainL,Y_testL = train_test_split(X,Y,test_size=0.3)
#check
X_trainL.shape,X_testL.shape,Y_trainL.shape,Y_testL.shape

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
logis = LogisticRegression()
#use function, fit, train the parameter 
logis.fit(X_trainL, Y_trainL)
#prediction
predicted = logis.predict(X_testL)
expected = Y_testL

In [ ]:
#score
logis_score_train = logis.score(X_trainL,Y_trainL)
print('Training score:',logis_score_train)
logis_score_test = logis.score(X_test,Y_test)
print('Testing score:',logis_score_test)


In [ ]:
# Decision Tree
X_trainD,X_testD,Y_trainD,Y_testD = train_test_split(X,Y,test_size=0.3)
#check
X_trainD.shape,X_testD.shape,Y_trainD.shape,Y_testD.shape
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=3)
# decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_trainD, Y_trainD)
# Y_pred_7 = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_trainD, Y_trainD) * 100, 2)
acc_decision_tree

In [ ]:
from sklearn import tree
import graphviz 
dot_data = tree.export_graphviz(decision_tree, out_file=None, 
                                feature_names = X_trainD.columns.tolist(), class_names = True,
                                filled = True, rounded = True)
graph = graphviz.Source(dot_data) 
graph